In [1]:
#Import Dependencies
import pandas as pd
import time
from bs4 import BeautifulSoup #as bs
import requests as req
from splinter import Browser
from selenium import webdriver
import pymongo


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.new_wine_store_db
collection = db.items

In [12]:
db.items.find().count()

4769

In [13]:
# clear test run data from mongodb 
# collection.delete_many();
# db.items.delete_many({});
db.items.find().count()

0

In [14]:
def scrape_wine():
    browser = webdriver.Chrome('chrome/chromedriver.exe')
    news = {}
    mainurl = 'https://www.binnys.com' 
    
    start = "https://www.binnys.com/wine/"
    
    browser.get(start)
    #time.sleep(1)
    
    html = browser.page_source 
    soup = BeautifulSoup(html, "html.parser")

    pages = int(522)
    
       
    for page in range(1, pages):
                # Error handling
        try:

            print(page)
            if page == 1:
                url = start
                browser.get(url)
                #time.sleep(1)

                html = browser.page_source 
                soup = BeautifulSoup(html, "html.parser")

#                 alldivs = ''
                alldivs = soup.find_all('div',class_='products wrapper container grid products-grid')
        
#                 print("get all divs")
        
#                 print(alldivs)

                for alldiv in alldivs:
#                     print("in first alldiv loop ")
#                     alllis = ''
                     
                    alllis = soup.find_all('li',class_='item product product-item col-md-3 col-sm-6')
                    
#                     print('get alllis')
            
#                     print(alllis)
                    
                    for allli in alllis:
#                         print('in allli loop')
            
                        # Get Title
                        title = allli.find("a", class_="product-item-link").get_text()
                       
                        print("title " + title)
#                         title_new = replace(title,'\n','')
#                         print(title_new)
                        
                        # Get Image URL for Photo 
                      
                        if str(allli.find('img', class_="photo image lazy loaded")) == 'None':
                            if str(allli.find('img', class_="photo image lazy")) == 'None':
                                print('no imageurl')
                            else:
                                image = allli.find('img', class_="photo image lazy")
                                imageurl = image['data-src']
#                                 print("found imageurl " + imageurl)  
                        elif allli.find('img',class_='photo image lazy loaded') != 'None': 
                                image = allli.find('img', class_="photo image lazy loaded")
                                imageurl = image['data-src']
#                                 print("found loaded imageurl " + imageurl)

                        
#                         print("before rating code")


                        # Get Reviewer points for wine 
                        if str(allli.find('span', class_="rating-reviewer__range")) == 'None':
                            print("no rating " + str(allli.find('span', class_="rating-reviewer__range")))
                        else:    
                            points = allli.find('span', class_="rating-reviewer__range").get_text()
#                             print("points " + points)
                        
#                         # Get Image URL for Photo 
#                         if str(allli.find('img', class_="photo image lazy loaded")) == 'None':
#                             print("no imageurl " + str(allli.find('img', class_="photo image lazy loaded")))
#                         else:    
#                             image = allli.find('img', class_="photo image lazy loaded")
#                             imageurl = image['src']
#                             print("imageurl " + imageurl)
                        
#                         print("before wine url code ")
                        #Get url for wine page             
                        if str(allli.find('a', class_="product-item-link")) == 'None':
                            print("no wine url " + str(allli.find('a', class_="product-item-link")))
                        else:    
                            winelink = allli.find('a', class_="product-item-link")
#                             wineurl = mainurl + winelink['href']
                            wineurl = winelink['href']
#                             print("wineurl " + wineurl)


#                         print("before wine price code ")
                        #Get price for wine              
                        if str(allli.find('span', class_="price")) == 'None':
                            print("no price " + str(allli.find('span', class_="price")))
                        else:
                            price = allli.find('span', class_="price").get_text()
#                             print("price " + price)


#                         print("before size code")
                        # Get Size 
                        if str(allli.find('div', class_="size-pack-attr")) =="None":
                            print("no size " + allli.find('div', class_="size-pack-attr"))
                        else:
                            size = allli.find('div', class_="size-pack-attr").get_text()
#                             print(size)

##### go to wine page to get description 

#                         print("before going to wine page ")
                        url2 = wineurl
                        browser.get(url2)
                        html = browser.page_source 
                        soup = BeautifulSoup(html, "html.parser")
                
#                         alldivs_wine = soup.find_all('div',class_='page-wrapper')
#                         alldivs_wine = ''

                        alldivs_wine = soup.find_all('div',class_='page-wrapper')
#                         print(alldivs_wine)
#                         print(url2)
#                         print("got alldivs_wine ")
#                       # Get Description 
                        for alldiv_wine in alldivs_wine:
                        
#                             print("in alldiv_wine loop ")
                
#                             print("before description code ")
                            # Get Description 
                            if str(alldiv_wine.find('div', class_="description")) == 'None':
                                print("no description " + soup.find('div',class_='description'))                
                            else:
                                description = alldiv_wine.find('div', class_='description').get_text()
#                                 print(description)
                            
#                             print("before attributes code ")
#                             allattributes = ''
                            # Get Type, Varietal, Area
                            allattributes = alldiv_wine.find_all('ul', class_="product-container-attributes")
#                             print('attributes ' + str(allattributes))
                            
#                             for allattribute in allattributes:
 
#                             allattrlis = ''
                            allattrlis = soup.find_all('li',class_='product attribute ')
                            
#                             print("got all allattrlis ")
    
        #                     print(alllis)

                            for allattrli in allattrlis:
#                                 print("in allattrli loop ")
                    
                                attrtype = allattrli.find("span", class_="type").get_text()    
                                attrvalue = allattrli.find('span', class_='value').get_text()
        #                                 for alltype in alltypes:
#                                 print(str(attrtype)+": " + str(attrvalue))
                                attributes = "'" + attrtype.strip() + " '" + ':' + attrvalue.strip() 
#                                 print(attributes)

               

                        
                            
#                             print("before post")
# #                     Dictionary to be inserted as a MongoDB document
                            post = {
                                'title': title,
                                'image': imageurl,
                                'wineurl': wineurl,
                                'points': points,
                                'price': price,
                                'size': size,
                                'description': description,
                                'attributes': attributes
                                }


                            collection.insert_one(post)

    #                         print("after insert")
        
#################################        
########### page 2 and beyond
#################################

            else:
#                 print('in else')
                url = "https://www.binnys.com/wine?p=" + str(page)
#                 print(url)
                browser.get(url)

                html = browser.page_source 
                soup = BeautifulSoup(html, "html.parser")


                alldivs = ''
                alldivs = soup.find_all('div',class_='products wrapper container grid products-grid')

          #                 print(alldivs)

                for alldiv in alldivs:
#                     print("in all divs")
                    # Get Wine name 
#                     alllis = ''    
                    alllis = soup.find_all('li',class_='item product product-item col-md-3 col-sm-6')
#                     print(alllis)
                    
                    for allli in alllis: 
                        title = allli.find("a", class_="product-item-link").get_text()
                       
                        print("title " + title)
#                         title_new = replace(title,'\n','')
#                         print(title_new)

                        # Get Reviewer points for wine 
                        if str(allli.find('span', class_="rating-reviewer__range")) == 'None':
                            print("no points " + str(allli.find('span', class_="rating-reviewer__range")))
                        else:    
                            points = allli.find('span', class_="rating-reviewer__range").get_text()
                            print("points " + points)
                        

#                         # Get Image URL for Photo 
                        if str(allli.find('img', class_="photo image lazy loaded")) == 'None':
                            if str(allli.find('img', class_="photo image lazy")) == 'None':
                                print('no imageurl')
                            else:
                                image = allli.find('img', class_="photo image lazy")
                                imageurl = image['data-src']
#                                 print("found imageurl " + imageurl)  
                        elif allli.find('img',class_='photo image lazy loaded') != 'None': 
                                image = allli.find('img', class_="photo image lazy loaded")
                                imageurl = image['data-src']
#                                 print("found loaded imageurl " + imageurl)


                        


                        #Get url for wine page             
                        if str(allli.find('a', class_="product-item-link")) == 'None':
                            print("no wine url " + str(allli.find('a', class_="product-item-link")))
                        else:    
                            winelink = allli.find('a', class_="product-item-link")
#                             wineurl = mainurl + winelink['href']
                            wineurl = winelink['href']
#                             print("wineurl " + wineurl)



                        #Get price for wine              
                        if str(allli.find('span', class_="price")) == 'None':
                            print("no price " + str(allli.find('span', class_="price")))
                        else:
                            price = allli.find('span', class_="price").get_text()
#                             print("price " + price)

                        # Get Size 
                        if str(allli.find('div', class_="size-pack-attr")) =="None":
                            print("no size " + allli.find('div', class_="size-pack-attr"))
                        else:
                            size = allli.find('div', class_="size-pack-attr").get_text()
#                             print(size)

##### go to wine page to get description 

                        url2 = wineurl
#                         print(url2)
                        browser.get(url2)
                        html = browser.page_source 
                        soup = BeautifulSoup(html, "html.parser")
                
#                         alldivs_wine = soup.find_all('div',class_='page-wrapper')

                        alldivs_wine = soup.find_all('div',class_='page-wrapper')
#                         print(alldivs_wine)
#                         print(url2)

#                       # Get Description 
                        for alldiv_wine in alldivs_wine:
        
                            # Get Description 
                            if str(alldiv_wine.find('div', class_="description")) == 'None':
#                                 print("no description " + soup.find('div',class_='description'))  
                                description = ''
                            else:
                                description = alldiv_wine.find('div', class_='description').get_text()
#                                 print(description)

                                  
                        # Get Type, Varietal, Area
                            allattributes = alldiv_wine.find_all('ul', class_="product-container-attributes")
#                             print('attributes ' + str(allattributes))
                            
#                             for allattribute in allattributes:
 
#                             allattrlis = ''
                            allattrlis = soup.find_all('li',class_='product attribute ')

        #                     print(alllis)

                            for allattrli in allattrlis: 
                                if allattrli.find("span", class_="type") == 'None':
                                    print("no type")
                                else:
                                    attrtype = allattrli.find("span", class_="type").get_text()
                                
                                if allattrli.find('span', class_='value') == 'None':
                                    print("no value")
                                else:
                                    attrvalue = allattrli.find('span', class_='value').get_text()
        #                                 for alltype in alltypes:
#                                 print(str(attrtype)+": " + str(attrvalue))
                                
                                attributes = "'" + attrtype.strip() + " '" + ':' + attrvalue.strip() 
#                                 print(attributes)

# #                     Dictionary to be inserted as a MongoDB document
                            post = {
                                'title': title,
                                'image': imageurl,
                                'wineurl': wineurl,
                                'points': points,
                                'price': price,
                                'size': size,
                                'description': description,
                                'attributes': attributes
                                }

                            collection.insert_one(post)
      
        except Exception as e:
            print(e)        


scrape_wine()




1
title 
                                    Kendall Jackson Vintner's Reserve Chardonnay 2016                                
title 
                                    Kim Crawford Sauvignon Blanc 2017                                
title 
                                    Meiomi Pinot Noir 2016                                
title 
                                    Arani Sauvignon Blanc 2017                                
no rating None
title 
                                    Bellevino Moscato d'Asti                                
no rating None
title 
                                    Bellevino Sweet Rosso                                
no rating None
title 
                                    Oyster Bay Sauvignon Blanc 2017                                
title 
                                    Menage a Trois Red 2016                                
no rating None
title 
                                    Liparita Cabernet Sauvignon Napa Valley 2014              

title 
                                    Concha y Toro Frontera Chardonnay                                
no points None
title 
                                    Hess Select Cabernet Sauvignon 2015                                
points 90
4
title 
                                    Liparita Napa Sauvignon Blanc 2017                                
no points None
title 
                                    Cupcake Pinot Grigio 2016                                
no points None
title 
                                    Los Vascos Cabernet Sauvignon                                
points 91
title 
                                    DAOU Winery Cabernet Sauvignon 2016                                
points 91-92
title 
                                    Robert Mondavi Private Selection Cabernet Sauvignon 2016                                
no points None
title 
                                    Stella Rosa Rosso by il Conte                                
points 85
title 
    

title 
                                    Roscato Rosso Dolce                                
no points None
title 
                                    14 Hands Hot to Trot Red Blend 2015                                
no points None
title 
                                    Pommery Brut Royal                                
points 90
title 
                                    Tres Lagunas Garnacha 2016                                
no points None
title 
                                    Yellow Tail Moscato                                
points 87
title 
                                    Korbel Brut                                
points 87
title 
                                    Bogle Old Vine Zinfandel 2015                                
no points None
7
title 
                                    Cavit Pinot Grigio                                
no points None
title 
                                    Josh Cellars Cabernet Sauvignon 2016                               

title 
                                    Domaine Santa Duc Cotes du Rhone Vieilles Vignes 2015                                
no points None
title 
                                    Coppola Diamond Collection Chardonnay Gold Label 2016                                
no points None
title 
                                    Barefoot Pink Moscato                                
no points None
title 
                                    Conundrum 2015                                
no points None
title 
                                    Ruffino Chianti Classico Riserva Ducale 2014                                
no points None
title 
                                    King Estate Pinot Gris 2016                                
points 91
title 
                                    Woodbridge Sauvignon Blanc                                
no points None
title 
                                    Casa Santos Lima Colossal Red Blend 2015                                
points 90
titl

title 
                                    Frank Family Cabernet Sauvignon 2014                                
points 91
title 
                                    Mezzacorona Pinot Grigio                                
no points None
title 
                                    Domaine de Pouy Gascogne Blanc 2016                                
no points None
title 
                                    Domaine Lafage Cuvee Nicolas 2015                                
points 92
title 
                                    Sutter Home Chardonnay 4 Pack                                
no points None
title 
                                    Pomelo Sauvignon Blanc 2016                                
no points None
title 
                                    Sutter Home Cabernet Sauvignon 4 Pack                                
no points None
title 
                                    Barefoot Moscato                                
no points None
title 
                                    Ra

...

title 
                                    Ca dei Frati Ronchedone Rosso 2014                                
no points None
title 
                                    Juris Burgenland St Laurent Selection 2015                                
no points None
title 
                                    Ransom Cabernet Franc 2015                                
no points None
title 
                                    Kanonkop Kadette 2016                                
no points None
title 
                                    Argento Malbec 2016                                
no points None
title 
                                    Giorgio Pelissero Dolcetto d'Alba Munfrina 2015                                
no points None
title 
                                    Mas Buscados Tempranillo Petit Verdot 2016                                
no points None
title 
                                    Fess Parker The Big Easy Red Blend 2016                                
no points None
ti

title 
                                    Josh Cellars Chardonnay 2016                                
no points None
title 
                                    Eisele Vineyard Sauvignon Blanc by Araujo Estate 2015                                
no points None
title 
                                    Hill Prairie Winery Peach Sangria                                
no points None
title 
                                    Hill Prairie Winery Prairie Barn Sweet Red                                
no points None
title 
                                    Hill Prairie Winery Adam's Apple                                
no points None
title 
                                    Broadley Vineyards Pinot Noir Estate 2016                                
no points None
title 
                                    Wyldewood Wylde Valley Red                                
no points None
title 
                                    Louis Guntrum Dry Riesling 2016                                
n

title 
                                    Merani Khvanchkara Semi-Sweet Red                                
no points None
title 
                                    Bura Mrgudic Plavac Mali Galerija Dingac 2012                                
no points None
title 
                                    Psagot Merlot 2013                                
no points None
title 
                                    Furlan Castelcosa Tocai Friulano Mari Me 2013                                
no points None
title 
                                    Perrier Jouet Belle Epoque 2 Flute Gift Set 2008                                
no points None
title 
                                    Michel Gonet Blanc de Blancs Prestige 2010                                
no points None
title 
                                    Williams Selyem Pinot Noir Sonoma Coast 2013                                
no points None
515
title 
                                    Anne Amie Cuvee A Muller Thurgau 2016    

518
title 
                                    Anne Amie Cuvee A Muller Thurgau 2016                                
no points None
title 
                                    Tabor Adama Cabernet Sauvignon 2013                                
no points None
title 
                                    Chateau Maris Rose in a Can                                
no points None
title 
                                    Vinum Regum Tokaji Aszu 5 Puttonyos Sweet White 2008                                
no points None
title 
                                    Josh Cellars Chardonnay 2016                                
no points None
title 
                                    Eisele Vineyard Sauvignon Blanc by Araujo Estate 2015                                
no points None
title 
                                    Hill Prairie Winery Peach Sangria                                
no points None
title 
                                    Hill Prairie Winery Prairie Barn Sweet Red         

title 
                                    Hill Prairie Winery Adam's Apple                                
no points None
title 
                                    Broadley Vineyards Pinot Noir Estate 2016                                
no points None
title 
                                    Wyldewood Wylde Valley Red                                
no points None
title 
                                    Louis Guntrum Dry Riesling 2016                                
no points None
title 
                                    Merani Khvanchkara Semi-Sweet Red                                
no points None
title 
                                    Bura Mrgudic Plavac Mali Galerija Dingac 2012                                
no points None
title 
                                    Psagot Merlot 2013                                
no points None
title 
                                    Furlan Castelcosa Tocai Friulano Mari Me 2013                                
no points None
t

In [ ]:
for post in db.items.find():
    print(post)

In [15]:
db.items.find().count()
# 12301
#12425

12425

In [ ]:
for doc in db.items.find().sort('title', pymongo.ASCENDING):
    print(doc)

In [ ]:
# Search  FUll Title
# this one works - full title !!!!!!!
list(db.items.find({'title':'\nBeringer California Collection Merlot\n'}))


In [19]:
# Search Fuzzy Match 
# This works Fuzzy Match

# list(db.items.find({'title': {'$regex': 'Chardon'}}))

# list(db.items.find({'description': {'$regex': 'fruit'}}))

list(db.items.find({'attributes': {'$regex': 'Cabernet'}}))

[{'_id': ObjectId('5b44af9bbb4c982c6cd407e5'),
  'attributes': "'Varietal ':Cabernet Sauvignon",
  'description': '\n        Cabernet Sauvignon offers character, complexity, and longevity, while Merlot adds softer tannins and elegance. The wine is fruity with aromas of chocolate, black plums and vanilla. A medium body gives way to black cherry flavors, soft tannins and a pleasing finish. Reviewed by: Producer  ',
  'image': 'https://www.binnys.com/media/catalog/product/cache/small_image/300x300/beff4985b56e3afdbeabfc89641a4582/4/0/400585.jpg',
  'points': '89',
  'price': '$8.99',
  'size': '\n                                                                    1.5 L Bottle                                                            ',
  'title': '\n                                    Concha y Toro Frontera Cabernet Merlot                                ',
  'wineurl': 'https://www.binnys.com/concha-y-toro-frontera-cabernet-merlot-400585.html'},
 {'_id': ObjectId('5b44afe5bb4c982c6cd4080